### TextBlob 라이브러리
- 익숙한 인터페이스를 통해 일반적인 텍스트 처리 작업에 대한 액세스를 제공
- TextBlob 객체를 자연어 처리를 수행하는 방법을 학습 한 Python 문자열 인 것처럼 처리 할 수 있다.
- 참조: https://textblob.readthedocs.io/en/latest/quickstart.html#translation-and-language-detection

In [2]:
from textblob import TextBlob as tb
import nltk
import pandas as pd
import numpy as np

In [3]:
txt1=tb('맥주가 좋았습니다.')
txt2=tb('Beer was good.')

In [4]:
#### TextBlob를 이용한 언어 번역
print(txt1.translate(to='en'))
print(txt2.translate(to='ja'))
print(txt2.translate(to='ko'))

Beer was good.
ビールはよかったです。
맥주는 좋았습니다.


In [5]:
#nltk.download()

### 텍스트 감정 분류기 만들기
- 학습 데이터(train): 문장과 긍정/부정
- 학습데이터를 이용해 학습하고 문자의 긍정/부정을 분류

In [6]:
from textblob.classifiers import NaiveBayesClassifier

# 학습 데이터 세트
train = [ ('I love this sandwich.', 'pos'), 
        ('This is an amazing place!', 'pos'), 
        ('I feel very good about these beers.', 'pos'), 
        ('This is my best work.', 'pos'), 
        ("What an awesome view", 'pos'), 
        ('I do not like this restaurant', 'neg'), 
        ('I am tired of this stuff.', 'neg'), 
        ("I can't deal with this", 'neg'), 
        ('He is my sworn enemy!', 'neg'), 
        ('My boss is horrible.', 'neg') ]

In [7]:
## 테스트 세트
test = [('The beer was good.', 'pos'),
        ('I do not enjoy my job', 'neg'),
        ("I ain't feeling dandy today.", 'neg'),
        ("I feel amazing!", 'pos'),
        ('Gary is a friend of mine.', 'pos'),
        ("I can't believe I'm doing this.", 'neg')]

In [8]:
### NaiveBayesClassifier 개체에 학습 데이터를 입력하고 긍정(pos)/부정(neg) 모델 생성
pos_neg=NaiveBayesClassifier(train)
pos_neg

<NaiveBayesClassifier trained on 10 instances>

In [9]:
pos_neg.classify("I ain't feeling dandy today")

'neg'

In [10]:
PosNeg_lst = []

for w in test:
    clsss = pos_neg.classify(w[0])
    PosNeg_lst.append((w[-1], clsss))
    
pd.DataFrame(PosNeg_lst, columns = ['예측', '정답'])

,예측,정답
0,pos,pos
1,neg,neg
2,neg,neg
3,pos,pos
4,pos,neg
5,neg,neg


In [11]:
print(f"정확도: {5/6}")

정확도: 0.8333333333333334


In [12]:
# 긍정에 대한 정밀도
print(f"긍정 정밀도: {2/2}")

긍정 정밀도: 1.0


In [13]:
# 부정에 대한 정밀도
print(f"긍정 정밀도: {3/4}")

긍정 정밀도: 0.75


In [14]:
# TextBlob 객체를 사용하여 여러 문장으로 구성된 텍스트도 분류
tag2 = tb("The beer was amazing. But the hangover was horrible. My boss was not happy.", classifier = pos_neg)
print(tag2.classify())


neg


In [15]:
## tag2을 분리해서 긍/부정 평가
print(tag2.sentences)

for sen in tag2.sentences:
    print(sen, "/", sen.classify())

[Sentence("The beer was amazing."), Sentence("But the hangover was horrible."), Sentence("My boss was not happy.")]
The beer was amazing. / pos
But the hangover was horrible. / neg
My boss was not happy. / neg


In [16]:
# 테스트 데이터를 이용해 정확도 계산
acc = pos_neg.accuracy(test)
print(acc)

0.8333333333333334


In [17]:
# show_informative_features를 실행하여 분류기가 어떤 기준으로 분류하는지 확인
print(pos_neg.show_informative_features())

Most Informative Features
          contains(this) = True              neg : pos    =      2.3 : 1.0
          contains(this) = False             pos : neg    =      1.8 : 1.0
          contains(This) = False             neg : pos    =      1.6 : 1.0
            contains(an) = False             neg : pos    =      1.6 : 1.0
             contains(I) = False             pos : neg    =      1.4 : 1.0
             contains(I) = True              neg : pos    =      1.4 : 1.0
            contains(He) = False             pos : neg    =      1.2 : 1.0
            contains(My) = False             pos : neg    =      1.2 : 1.0
          contains(What) = False             neg : pos    =      1.2 : 1.0
         contains(about) = False             neg : pos    =      1.2 : 1.0
None


### 한글 긍정/부정어 처리

In [18]:
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier

In [19]:
train = [('나는 이 샌드위치를 정말 좋아해.', '긍정'),
         ('정말 멋진 곳이에요!', '긍정'),
         ('나는 이 맥주들이 아주 좋다고 생각해요.', '긍정'),
         ('이것은 나의 최고의 작품입니다.', '긍정'),
         ("정말 멋진 광경이다", "긍정"),
         ('난 이 식당 싫어', '부정'),
         ('난 이게 지겨워.', '부정'),
         ("이 문제는 처리할 수 없습니다.", "부정"),
         ('그는 나의 불구대천의 원수이다.', '부정'),
         ('내 상사는 끔찍해.', '부정'),
         ('나는 내 꿈을 믿는다', '긍정'),
         ('나는 매일 최선을 다하고 있다', '긍정'),
         ('나는 있는 그대로의 나를 사랑한다', '긍정'),
         ('나는 내 삶을 100% 책임진다', '긍정'),
         ('가장 좋은 일은 아직 생기지 않았다', '긍정'),
         ('나는 매일 나의 삶에 감사한다', '긍정'),
         ('새로나온 휴대폰은 배터리 교체가 되지 않아 불편하다', '부정'),
         ('이번에 나온 영화 너무 재밌다. 주말에 또 보고 싶다.', '긍정'),
         ('나의 아버지는 이해가 안된다', '부정'),
         ('나는 어머니와 있을 때 퉁명해진다', '부정'),
         ('나는 어머니와 있을 때 불편할 때가 있다.', '부정'),
         ('치킨집에 선결제로 주문했는데 돈은 결제가 되었는데 치킨집에선 주문이 안들어왔다고합니다.', '부정'),
         ('지금 현재 네이버 실시간 검색어에 떴고, 여전히 고객센터는 전화연결이 되지않습니다. ', '부정'),
         ('이럴거면 요기요처럼 아예 접속도 안되게하던가 결제를 막았어야하는데', '부정'),
         ('결제해서 가게에 돈 준 것도 아니고 본인들이 가져가놓고 전화도 안받으면 돈은 어떻게 돌려받습니까?', '부정'),
         ('너무 유용하고 편리하네요.. ', '긍정'),
         ('처음 구입한다고 쿠폰 받고 이용하니깐 저렴하게 먹을 수 있었고 배달시간도 만족스러워서 좋았습니다.', '긍정'),
         ('배달의 민족 강추 합니다. ', '긍정'),
         ('자주 이용할께요ㅡㅡㅡ강추 강추!!!', '긍정') ]

In [20]:
test = [('맥주가 좋았습니다.', '긍정'),
        ('난 내 일을 즐기지 않는다', '부정'),
        ('오늘은 기분이 안 좋아요.', '부정'),
        ('빠른 배달 좋네요', '긍정'),
        ('네드는 나의 친구입니다.', '긍정'),
        ('제가 이렇게 하고 있다니 믿을 수가 없어요.', '부정')]

In [21]:
pos_neg = NaiveBayesClassifier(train)
pos_neg

<NaiveBayesClassifier trained on 29 instances>

In [22]:
acc = pos_neg.accuracy(test)
print(acc)

0.5


### 배민 앱 사용 후기를 이용한 긍부정 모델 생성

In [23]:
import pandas as pd
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier

df = pd.read_csv('./data/배민어플후기.csv',encoding = 'cp949')
df.head()

,Reviews,Unnamed: 1
0,어이가없어서말도안나오네요 받지도않은음식을결제하라는게말이되는건가요? 출근도해야하는데 ...,긍정
1,제 동생 추천으로 주문해본건데 동생말이 틀리지 않았어요. 브랜드 친키보다 제 입맛엔...,긍정
2,뭐만하면 배달팁 추가...비많이 와서 그러는건 고생하시니 그러려니하는데 배달이 몰려...,긍정
3,지금사용중인데 배달주문해서 돈은빠져나갔는데 배달주문한곳에전화해서 확인해보니 주문들어...,긍정
4,"써본지는 얼마 안되지만 몇번 배달시켜 먹어본 결과, 편리하고 좋아요^^ 맛있는 맛집...",NaN


In [24]:
df.columns = ["Reviews", "pos_neg"]
df.head()

,Reviews,pos_neg
0,어이가없어서말도안나오네요 받지도않은음식을결제하라는게말이되는건가요? 출근도해야하는데 ...,긍정
1,제 동생 추천으로 주문해본건데 동생말이 틀리지 않았어요. 브랜드 친키보다 제 입맛엔...,긍정
2,뭐만하면 배달팁 추가...비많이 와서 그러는건 고생하시니 그러려니하는데 배달이 몰려...,긍정
3,지금사용중인데 배달주문해서 돈은빠져나갔는데 배달주문한곳에전화해서 확인해보니 주문들어...,긍정
4,"써본지는 얼마 안되지만 몇번 배달시켜 먹어본 결과, 편리하고 좋아요^^ 맛있는 맛집...",NaN


In [25]:
## NaN 값 확인
df.isna().sum()

Reviews      0
pos_neg    388
dtype: int64

In [26]:
df.shape

(4680, 2)

In [27]:
df_nan = df[df['pos_neg'].isnull()]
df_nan

,Reviews,pos_neg
4,"써본지는 얼마 안되지만 몇번 배달시켜 먹어본 결과, 편리하고 좋아요^^ 맛있는 맛집...",NaN
6,"언제 먹어도 맛있는족발 ,콜라겐 덩어리 ..맛있게 먹고.젊어지고..아미족발 굿 .....",NaN
34,"이벤트가 전부 선착순에 신규유저만 참여가능, 뽑기 등 혜택을 받고 싶어도 못받는 이...",NaN
35,"배달이 빨랐고 , 탕슉도 맛있어요. 탕슉안에 만두가 속이 알차요.\t\t\t\t\t...",NaN
43,"괜찮은 어플. 리뷰도 있고, 주문도 쉽고 빠름.배달시킬때마다 배민사용함\t\t\t\...",NaN
...,...,...
4675,아주편리하게 이용하고있습니다,NaN
4676,매번 이걸로만 시켜먹음,NaN
4677,대한민국 최고 배달어플,NaN
4678,쿠폰이벤트해서 정각되자마자 들어갔더니 시스템오류뜨더니 주문이 아예 안되네요,NaN


In [28]:
### df에서 NaN 값 제거
df = df.dropna()
df.isna().sum()

Reviews    0
pos_neg    0
dtype: int64

### train/test 데이터 생성하기
- train: 전체 데이터의 3/4 (75%)
- test : 전체 데이터의 1/4 (25%)

In [29]:
# 긍정/부정 데이터 개수 확인
df_pos = df[df["pos_neg"] == "긍정"]
df_neg = df[df["pos_neg"] == "부정"]

print(len(df_pos), len(df_neg))

784 3508


In [30]:
# 긍정과 부정 데이터 개수 동일하게 맞추기

df_neg = df_neg.sample(784)
print(len(df_pos), len(df_neg))

784 784


In [31]:
print("train:", int(784*0.75), "test:", 784 - int(784*0.75))

train: 588 test: 196


In [32]:
# train / test 데이터 준비
df_train = pd.concat([df_pos[:589], df_neg[:589]])
df_test = pd.concat([df_pos[589:], df_neg[589:]])

print(len(df_train),len(df_test))

1178 390


In [33]:
# 데이터 순서를 랜덤하게 만들기
df_train = df_train.sample(1178).reset_index(drop = True)
df_test = df_test.sample(390).reset_index(drop = True)
df_test.head(10)

,Reviews,pos_neg
0,12시까지 배달 부탁드렸는데 시간 맞춰서 따끈따끈 키친 배달해주셨네요 짱입니다 ^^,부정
1,배민은 괜히 배민이 아닌 거 같아요ㅠㅠㅠ넘 맛있는 배민 라이더스 맛집 찾아버렸고~ ...,긍정
2,편하고 좋아요7!,긍정
3,앱이 자꾸 튕깁니다. 메뉴 선택하고 결제하러 넘어가기 직전에서(초록색 탭을 클릭하는...,부정
4,다른 배달앱과 비교해서 나쁘지 않음,부정
5,배민 쏜다쿠폰보고 탈퇴삭제합니다. 내가 미친색히지.ㅡㅡ,부정
6,배민으로 시켜먹으면서 제일 큰 금액으로 받아본 쿠폰이 선착순으로 겨우 받은 만원짜리...,부정
7,유용하게 잘쓰고 있는데요 지난 주문내역을 삭제할수있게 해주세요 때론 지우고싶은 내역...,긍정
8,빠르고 조아요~~~,긍정
9,아니 이거 뭐져 쿠폰으로 음식을 시켰는데 돈은 빠져나가고 주문현황에는 아무곳도 안뜨...,부정


In [34]:
# 튜플 구조로 변경
def tup_che(df_t):
    tr_te = []
    for i in range(len(df_t)):
        tr_te.append((df_t.iloc[i, 0], df_t.iloc[i, 1]))
    return tr_te



In [35]:
train = tup_che(df_train)
test = tup_che(df_test)

train[:5]

[('잘사용하고있어요 ^^', '부정'),
 ('첫사용5천원쿠폰 알차게 잘 썼습니다. 주문하기도 편하고 이런저런 이벤트도 타사보다 많네요. 오늘 잘 먹었습니다^^', '긍정'),
 ('다른 배달 어플보다 쿠폰을 더 자주 많이 주니까 좋아요', '부정'),
 ('좋아서 잘 쓰고 있어요^^', '부정'),
 ('쿠폰사논게 있어서 쓰려고 주문하기 하니 계속 오류네요ㅜㅜ 주문하기로 넘어가질 않아요ㅜㅜ', '긍정')]

In [38]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/wondongchan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier

pos_neg = NaiveBayesClassifier(train)
test_acc = pos_neg.accuracy(test)

print(f'정확도: {test_acc}')

정확도: 0.6743589743589744


In [46]:
df_nan

,Reviews,pos_neg
0,"써본지는 얼마 안되지만 몇번 배달시켜 먹어본 결과, 편리하고 좋아요^^ 맛있는 맛집...",NaN
1,"언제 먹어도 맛있는족발 ,콜라겐 덩어리 ..맛있게 먹고.젊어지고..아미족발 굿 .....",NaN
2,"이벤트가 전부 선착순에 신규유저만 참여가능, 뽑기 등 혜택을 받고 싶어도 못받는 이...",NaN
3,"배달이 빨랐고 , 탕슉도 맛있어요. 탕슉안에 만두가 속이 알차요.\t\t\t\t\t...",NaN
4,"괜찮은 어플. 리뷰도 있고, 주문도 쉽고 빠름.배달시킬때마다 배민사용함\t\t\t\...",NaN
...,...,...
383,아주편리하게 이용하고있습니다,NaN
384,매번 이걸로만 시켜먹음,NaN
385,대한민국 최고 배달어플,NaN
386,쿠폰이벤트해서 정각되자마자 들어갔더니 시스템오류뜨더니 주문이 아예 안되네요,NaN


In [43]:
df_nan = df_nan.reset_index(drop=True)

In [47]:
df_nan.head()

,Reviews,pos_neg
0,"써본지는 얼마 안되지만 몇번 배달시켜 먹어본 결과, 편리하고 좋아요^^ 맛있는 맛집...",NaN
1,"언제 먹어도 맛있는족발 ,콜라겐 덩어리 ..맛있게 먹고.젊어지고..아미족발 굿 .....",NaN
2,"이벤트가 전부 선착순에 신규유저만 참여가능, 뽑기 등 혜택을 받고 싶어도 못받는 이...",NaN
3,"배달이 빨랐고 , 탕슉도 맛있어요. 탕슉안에 만두가 속이 알차요.\t\t\t\t\t...",NaN
4,"괜찮은 어플. 리뷰도 있고, 주문도 쉽고 빠름.배달시킬때마다 배민사용함\t\t\t\...",NaN


In [53]:
pos_neg.classify(df_nan.loc[0, 'Reviews'])

'긍정'

In [56]:
# 데이터 정규식 표현으로 데이터 정리
# 긍정/부정 값 pos_neg 필드에 추가

import re

txt_pos_neg = []

for i in df_nan.index:
    text = re.sub('[\t]',"",df_nan.loc[i, 'Reviews'])
    txt_pos_neg.append(pos_neg.classify(text))

df_nan['pos_neg'] = txt_pos_neg
df_nan

,Reviews,pos_neg
0,"써본지는 얼마 안되지만 몇번 배달시켜 먹어본 결과, 편리하고 좋아요^^ 맛있는 맛집...",긍정
1,"언제 먹어도 맛있는족발 ,콜라겐 덩어리 ..맛있게 먹고.젊어지고..아미족발 굿 .....",긍정
2,"이벤트가 전부 선착순에 신규유저만 참여가능, 뽑기 등 혜택을 받고 싶어도 못받는 이...",긍정
3,"배달이 빨랐고 , 탕슉도 맛있어요. 탕슉안에 만두가 속이 알차요.\t\t\t\t\t...",긍정
4,"괜찮은 어플. 리뷰도 있고, 주문도 쉽고 빠름.배달시킬때마다 배민사용함\t\t\t\...",긍정
...,...,...
383,아주편리하게 이용하고있습니다,긍정
384,매번 이걸로만 시켜먹음,긍정
385,대한민국 최고 배달어플,긍정
386,쿠폰이벤트해서 정각되자마자 들어갔더니 시스템오류뜨더니 주문이 아예 안되네요,부정


In [62]:
# 배민 어플에 대한 긍부정 평가
df_nan.groupby('pos_neg')[["pos_neg"]].count()

,pos_neg
pos_neg,
긍정,149
부정,239


In [37]:
# model save
import joblib
joblib.dump(pos_neg, 'pos_neg_20220412.pkl')

['pos_neg_20220412.pkl']